In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# For reproducibility
torch.manual_seed(1)

## Discrete Probability Distribution
- point 자체가 확률이 됨. (연속확률분포에서는 point 자체가 아니라 어떤 범위가 확률이 되는 것)


## Softmax

P(주먹|가위), P(가위|가위), P(보|가위)를 알 수 있다면 상대가 가위를 냈을 때 그 다음에 무엇을 낼지 예측해볼 수 있음. 이러한 정보를 바탕으로 해당 확률 분포의 pdf를 근사해볼 수 있음.

- 일반적인 argmax의 방식으로는 max인 element에는 1, 그렇지 않은 경우에는 0이 되지만 softmax는 각각의 element들의 값이 합쳐서 1이 되게끔 구해질 것임.

In [ ]:
z = torch.FloatTensor([1, 2, 3])

hypothesis = F.softmax(z, dim = 0)
print(hypothesis) # 확률 값을 갖게 됨.

tensor([0.0900, 0.2447, 0.6652])


## Cross Entropy
- Loss function으로 사용함.
- 두 개의 확률분포가 주어졌을 때, 두 확률 분포가 얼마나 비슷한지 나타내주는 값.
P에서 sampling한 x 값을 Q에 대입하여 log를 취한 후 합한 값을 나타냄.
- Cross Entropy를 minimize하면 원하는 확률 분포로 근사할 수 있음.

In [ ]:
z = torch.rand(3, 5, requires_grad = True) # 5 classes, 3 samples
hypothesis = F.softmax(z, dim=1)
print(hypothesis) # prediction : y_hat

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [ ]:
# generate answer randomly
# 각각의 sample에 대해 정답 : index 0, 2, 1이 정답이 될 것임.
y = torch.randint(5, (3, )).long()
print(y)

tensor([0, 2, 1])


In [ ]:
# one hot vector로 만들어주기
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [ ]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


위 과정을 torch에서 제공해주는 `log_softmax()`로 쉽게 수행할 수 있다.

In [ ]:
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [ ]:
# 위의 cost function을 다음과 같이 바꿀 수 있다.
cost = (y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

In [ ]:
# 위보다 더 쉽게 작성하려면 nll_loss 사용하여 sum과 mean을 생략할 수 있음.
# nll : negative log likelihood loss
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [ ]:
# F.cross_entropy를 사용하면 F.log_softmax()와 F.nll_loss()를 한 번에 작동할 수 있음.
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

`cross_entropy()`가 간편하긴 하지만, neural network에서는 softmax의 값 이전 값을 사용해서 prediction 확률 값을 구해야할 수도 있어서, 필요에 따라서 알맞은 것을 사용해야함.

## Training with Low-level Cross Entropy Loss

In [ ]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]] # dim = 4
y_train = [2, 2, 2, 1, 1, 1, 0, 0] # one hot vector에서 1이 있는 위치
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

## High level implementation with nn.Module

In [ ]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3) # Output이 3!

  def forward(self, x):
    return self.linear(x)
    
model = SoftmaxClassifierModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # H(x) 계산
  prediction = model(x_train)

  # cost 계산
  cost = F.cross_entropy(prediction, y_train)

  # cost로 H(x)로 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 20번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609259
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922
